# cryoCARE odd-and_even registered of empiar10311

* Inputs: `even.mrc` and `odd.mrc`, the pair of noisy vols.
* Outputs: `odd_even_registered_denoised/empiar10311_stack_crop.mrc` with the denoised vol, and `denoise_odd_even_registered.pdf` with a view of a tile of a central slice in Z.

In [ ]:
from pathlib import Path
from collections import namedtuple

In [ ]:
Args = namedtuple("args", ["even", "odd", "denoised"])
args = Args("even.mrc", "odd.mrc", "odd_even_registered_denoised/empiar10311_stack_crop.mrc")

In [ ]:
if Path(args.denoised).exists():
    raise Exception(f"{args.denoised} already exists ... exiting")

In [ ]:
if not Path("even.mrc").exists() or not Path("odd.mrc"):
    %run split_even_odd.ipynb

In [ ]:
if not Path("even_registered.mrc").exists() or not Path("odd_registered.mrc"):
    %run project_odd_even_and_viceversa.ipynb

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

In [ ]:
# %pip install tensorflow==2.4

## Configure cryoCARE

%%writefile train_data_config__odd_even_registered.json
{
    "even": ["even_registered.mrc", "even.mrc"],
    "odd": ["odd.mrc", "odd_registered.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 800,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./odd_even_registered_data",
    "overwrite": "True"  
}

In [ ]:
%%writefile train_data_config__odd_even_registered.json
{
    "even": ["even.mrc", "odd_registered.mrc"],
    "odd": ["even_registered.mrc", "odd.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 800,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./odd_even_registered_data",
    "overwrite": "True"  
}

%%writefile train_data_config__odd_even_registered.json
{
    "even": ["even.mrc"],
    "odd": ["odd_registered.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 800,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./odd_even_registered_data",
    "overwrite": "True"  
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__odd_even_registered.json

## Train

In [ ]:
%%writefile train_config__odd_even_registered.json
{
  "train_data": "./odd_even_registered_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
source ~/envs/cryoCARE/bin/activate
cryoCARE_train.py --conf train_config__odd_even_registered.json

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='model.tar.gz',
    drive_file_name='empiar10311_stack_crop__denoised__cryoCARE_odd_even_registered__model.tar.gz',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

## Infer

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/empiar10311_stack_crop.mrc .

In [ ]:
%%writefile predict_config__odd_even_registered.json
{¶
    "path": "./model.tar.gz",
    "even": ["empiar10311_stack_crop.mrc"], 
    "odd": ["empiar10311_stack_crop.mrc"],
    "n_tiles": [1,1,1],
    "output": "odd_even_registered_denoised",
    "overwrite": "True",
    "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
#pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__odd_even_registered.json || true

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='odd_even_registered_denoised/empiar10311_stack_crop.mrc',
    drive_file_name='empiar10311_stack_crop__denoised__cryoCARE_odd_even_registered.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
import mrcfile
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(path_file):
    return mrcfile.read(mrc_file_path)

In [ ]:
mrc_file_path = 'empiar10311_stack_crop.mrc'
original_vol = read_MRC(mrc_file_path)

In [ ]:
mrc_file_path = 'odd_even_registered_denoised/empiar10311_stack_crop.mrc'
denoised_vol = read_MRC(mrc_file_path)

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_vol.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_vol[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_vol[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
slice_idx = original_vol.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_vol[slice_idx, :256, :256].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_vol[slice_idx, :256, :256].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
slice_idx = original_vol.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_vol[slice_idx, 500:1000, 1000:1500].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_vol[slice_idx, 500:1000, 1000:1500].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_vol.shape[0]//2
plt.imshow(denoised_vol[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
figure(figsize=(16, 16))
slice_idx = original_vol.shape[0]//2
plt.imshow(original_vol[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_vol.shape[0]//2
plt.imshow(denoised_vol[slice_idx, 200:600, 200:600], cmap="gray")
plt.savefig("denoise_odd_even_registered.pdf")

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoise_odd_even_registered.pdf',
    drive_file_name='empiar10311_stack_crop__denoised__cryoCARE_odd_even_registered.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)